# Implementación del método de la máxima verosimilitud para la regresión logística


### Definir la función de entorno $L(\beta)$
$$\large{L(\beta) = \prod_{i=1}^n}P_i^{y_i}(1-P_i)^{1-y_i}$$

In [1]:
def likelihood (y, pi):
    import numpy as np
    total_prod = 1
    for i in range(len(y)):
        prod_in = np.where(y[i]==1,pi[i],1-pi[i])
        total_prod = total_prod*prod_in
    return total_prod

### Calcular las probabilidades para cada observación
$$\large{P_i = P(x_i) = \frac{1}{1+e^{-(\beta \cdot x_1)}}}$$
Recordar que $\alpha = \beta_0$

In [2]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

$$W = diag(P_i \cdot (1-P_1))_{i=1}^n$$

In [3]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)

    for i in range(n):
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

### Obtener la solución de la Función Logística
$$\beta_{n+1} = \beta_n - \frac{f(\beta_n)}{f'(\beta_n)}$$
$$f(\beta) = X(Y-P)$$
$$f'(\beta) = XWX^T$$

In [4]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    n_rows = np.shape(X)[0]
    bias = np.ones(n_rows).reshape(n_rows, 1)
    X_new = np.append(X, bias, axis = 1)
    n_cols = np.shape(X_new)[1]

    beta = np.zeros(n_cols).reshape(n_cols, 1)
    root_dif = np.array(range(1,n_cols+1)).reshape(n_cols, 1)
    iter_i = 10000
    # El limite es de la dif de cambio
    while (iter_i > limit):
        print(iter_i, ", ", limit)
        pi = logitprobs(X_new, beta)
        print(pi)
        W = findW(pi)
        print(W)

        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        print("Beta: ",beta, "\n Iter: ", iter_i)
        beta = beta + root_dif
        iter_i = np.sum(root_dif * root_dif)
        ll = likelihood(Y, pi)
    return beta

## Comprobación experimental

In [5]:
import numpy as np

In [6]:
X = np.array(range(10)).reshape(10,1)

In [7]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [8]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [9]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias,axis=1)

In [10]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [11]:
a = logistics(X,Y,0.00001)

10000 ,  1e-05
[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta:  [[0.]
 [0.]] 
 Iter:  10000
5.777190082644626 ,  1e-05
[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.82687453])]
[[0.07859404 0.         0.         0.    

# Con el paquete Stats

In [15]:
import statsmodels.api as sm

In [16]:
logit_model = sm.Logit(Y,X_new)

In [17]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [22]:
print(result.summary(),"\n\n",result.summary2())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Tue, 05 Jan 2021   Pseudo R-squ.:                  0.3596
Time:                        12:07:18   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                   0.02781
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6622      0.400      1.655      0.098      -0.122       1.446
const         -3.6956      2.289     -1.615      0.106      -8.182       0.791

                         Results: Logit
Model:     